In [1]:
from bs4 import BeautifulSoup

In [2]:
f = open('../data/prova_1500/article_0.html', 'r')
soup = BeautifulSoup(f.read(), 'html.parser')
f.close()

In [15]:
title = soup.find('title')
title.contents[0].strip()

'Fullmetal Alchemist: Brotherhood - MyAnimeList.net'

In the following cell I'll iterate over the divs with class 'spaceit_pad' that contain some spam containing the value of interest for as that are:

* anime_type: Type
* number of episodes: Episodes
* release and end: Aired
* number of members: Members
* score: Score
* users: (in the same spam of Score)
* rank: Ranked
* popularity: Popularity

In [49]:
from datetime import datetime


divs = soup.find_all('div', {"class": "spaceit_pad"})
from_interest = ['Episodes:','Aired:','Members:','Ranked:','Popularity:']
ret = dict()
for div in divs:
    content = div.contents
    tag = content[1].contents[0]
    if tag=='Score:':
        print('Score:')
        attr= {'itemprop':'ratingValue'}
        ret['score'] = float(div.find('span', attr).contents[0])
        attr = {'itemprop':'ratingCount'}
        ret['users'] = int(div.find('span', attr).contents[0])
    elif tag == 'Type:':
        ret['type'] = content[3].contents[0]
    elif tag in from_interest:
        val = content[2].strip()
        if val.startswith('#'):
            val=val[1:]
            ret[tag[:-1].lower()] = int(val)
        elif 'to' in val:
            start, end = val.split('to')
            start = start.strip()
            start = datetime.strptime(start, '%b %d, %Y')
            end = datetime.strptime(end.strip(), '%b %d, %Y')
            val = f'start: {start}, end: {end}'
            ret['start_date'] = start
            ret['end_date'] = end
        else:
            ret[tag[:-1].lower()] = int(val.replace(',',''))
print(ret)
    #print(div.contents)

Score:
{'type': 'TV', 'episodes': 64, 'start_date': datetime.datetime(2009, 4, 5, 0, 0), 'end_date': datetime.datetime(2010, 7, 4, 0, 0), 'score': 9.16, 'users': 1622384, 'ranked': 1, 'popularity': 3, 'members': 2675751}


The remaining fields are:
* **Synopsis** (to save as animeDescription): *String*
* **Related Anime** (to save as animeRelated): Extract all the related animes, but only keep unique values and those that have a hyperlink associated to them. *List of strings*.
* **Characters** (to save as animeCharacters): *List of strings*.
* **Voices** (to save as animeVoices): *List of strings*
* **Staff** 

They are contained in different structures of the right side of the page and will be retrieved in the following cells

In this cell I'll retrieve the description of the anime (synopsis)

In [56]:
synopsis = soup.find('p', {'itemprop':'description'}).contents[0]
ret['synopsis'] = synopsis

In this cell I'll retrieve the **distinct** related animes in the page that can be found in the table with class 'anime_detail_related_anime'

In [69]:
related_animes_table = soup.find('table', {'class': 'anime_detail_related_anime'})
animes = set()
for a in related_animes_table.find_all('a'):
    animes.add(a.contents[0])

list(animes)
ret['related_anime'] = list(animes)

In [91]:
divs = soup.find_all('div', {'class':"detail-characters-list clearfix"})
i = 1
staff = []
for td in divs[-1].find_all('td', {'class':'borderClass'}):
    if (i)==0:
        #print(td)
        a, small = td.find_all(['small', 'a'])
        staff.append([a.contents[0], small.contents[0].split(',')])

    i = (i+1)%2

In [93]:
ret['staff'] = staff

Now the only features of interest are voices and characters and they can be found in the other div of the class "detail-characters-list clearfix" and they belong to the same table but in two different columns

In [104]:
ch_vc = [[], []]
links = list(map(lambda x: x.contents[0], divs[0].find_all('a')))
i = 0
        #print(td)
for l in links:
    if l=='\n':
        continue
    ch_vc[i].append(l)
    i = (i+1)%2

ret['characters'] = ch_vc[0]
ret['voices'] = ch_vc[1]

In [105]:
ret

{'type': 'TV',
 'episodes': 64,
 'start_date': datetime.datetime(2009, 4, 5, 0, 0),
 'end_date': datetime.datetime(2010, 7, 4, 0, 0),
 'score': 9.16,
 'users': 1622384,
 'ranked': 1,
 'popularity': 3,
 'members': 2675751,
 'synopsis': "After a horrific alchemy experiment goes wrong in the Elric household, brothers Edward and Alphonse are left in a catastrophic new reality. Ignoring the alchemical principle banning human transmutation, the boys attempted to bring their recently deceased mother back to life. Instead, they suffered brutal personal loss: Alphonse's body disintegrated while Edward lost a leg and then sacrificed an arm to keep Alphonse's soul in the physical realm by binding it to a hulking suit of armor.",
 'related_anime': ['Fullmetal Alchemist: The Sacred Star of Milos',
  'Fullmetal Alchemist: Brotherhood Specials',
  'Fullmetal Alchemist: Brotherhood - 4-Koma Theater',
  'Fullmetal Alchemist'],
 'staff': [['Cook, Justin', ['Producer']],
  ['Yonai, Noritomo', ['Producer'